In [11]:
## datadam_sns_floating_population_score

# module
import psycopg2
import pandas as pd

# 기준월 (202201 ~ 202209)
dates = pd.date_range('2022-01',periods=9,freq='M').strftime("%Y%m")

In [12]:
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

In [13]:
# 시군구코드
aws_cursor.execute(
"""
select distinct substring(pnu, 1, 5) as sig_cd
from m1.pnu
order by 1
"""
)
aws_conn.commit()
rows = aws_cursor.fetchall()
df = pd.DataFrame(rows, columns = ['sig_cd'])
lawd_cd = list(df['sig_cd'])

In [ ]:
# INSERT
for date in dates:
    aws_cursor.execute(
    """
    insert into m2.datadam_sns_floating_population_score (
    with date as (
        select '""" + date + """'::text as base_ym
    )
    select (select base_ym from date) as base_ym
         , A.pnu
         , substring(A.pnu, 1, 8) as emd_cd
         , round((((case when ((B.cnt - C.cnt_avg) / C.cnt_std * 5 + 75) > 100 then 100 else ((B.cnt - C.cnt_avg) / C.cnt_std * 5 + 75) end) 
           + (case when ((D.f_tot - E.pop_avg) / E.pop_std * 5 + 75) > 100 then 100 else ((D.f_tot - E.pop_avg) / E.pop_std * 5 + 75) end))/2), 2) as score
         , sysdate::date as create_at
         , sysdate::date as update_at
         , 'du.Park' as work_user
    from (
        -- 모집단 가져오기 : KT매출 & PNU존재 & 건물존재
        select T1.pnu
        from (
            -- KT매출 존재
            select distinct substring(bd_mgt_sn, 1, 19) as pnu
            from m1.kt_bldg_sales
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
        ) T1
        -- PNU 존재
        join m2.cremao_land T2
            on T1.pnu = T2.pnu
        -- 건물 존재
        join (
            select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
            from m1.bld_rgst
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
        ) T3
            on T1.pnu = T3.pnu
    ) A
    join ( -- sns 노출 수 join (n:1 매칭) - B, C
        select *
        from m1.instagram_emd_cnt
    ) B
        on substring(A.pnu, 1, 8) = substring(B.emd_cd, 1, 8)
    left join (
        select avg(cnt) as cnt_avg
             , stddev(cnt::integer) as cnt_std
        from m1.instagram_emd_cnt
        where cnt > 0 and cnt notnull
    ) C
        on 1=1
    left join ( -- 유동인구 join (1:1 매칭) - D, E
        select substring(bd_mgt_sn, 1, 18) as pnu, avg(f_tot) as f_tot
        from m1.kt_bldg_xy_info
        where base_ym = (select base_ym from date) -- 기준월
        and f_tot > 0
        group by 1
    ) D
        on substring(A.pnu, 1, 18) = substring(D.pnu, 1, 18)
    left join (
        select avg(f_tot) 	 as pop_avg
             , stddev(f_tot) as pop_std
        from m1.kt_bldg_xy_info
        where base_ym = (select base_ym from date) -- 기준월
        and f_tot > 0
    ) E
        on 1=1
    );
    """
    )
    aws_conn.commit()
    print("Done")